# Logistic Regression and SGD Homework 
***
**Name**: $<$**Juan Lin**$>$ 
***

This assignment is due on Moodle by **5pm on Friday February 9th**. Submit only this Jupyter notebook to Moodle.  Do not compress it using tar, rar, zip, etc. Your solutions to analysis questions should be done in Markdown directly below the associated question.  Remember that you are encouraged to discuss the problems with your instructors and classmates, but **you must write all code and solutions on your own**.  For a refresher on the course **Collaboration Policy** click [here](https://github.com/chrisketelsen/CSCI5622-Machine-Learning/blob/master/resources/syllabus.md#collaboration-policy)



## Overview 
***


In this homework you'll implement stochastic gradient ascent for logistic regression and you'll apply it to the task of determining whether documents are talking about automobiles or motorcycles.

<br>

![autos_motorcycles](autos_motorcycles.jpg "A car and a motorcycle")


<br>

You should not use any libraries that implement any of the functionality of logistic regression for this assignment; logistic regression is implemented in Scikit-Learn, but you should do everything by hand now. You'll be able to use library implementations of logistic regression in the future.

Here are the rules: 

- Do **NOT** load or use any Python packages that are not available in Anaconda 3.6. 
- Some problems with code may be autograded.  If we provide a function or class API **do not** change it.
- Do not change the location of the data or data directory.  Use only relative paths to access the data. 

In [ ]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline 

### [5 points] Problem 1: Loading and Exploring the Data
***

The `Example` class will be used to store the features and labels associated with a single training or test example.  The `read_data` function will read in the text data and split it into training and test sets.  

 Load the data and then do the following: 
- Report the number of words in the vocabulary 
- Explain how the code is creating features (i.e. what text model is being used). 
- Go into the raw text files in the data directory and figure out which label (0/1) refers to which class of document (automobiles or motorcycles)

In [ ]:
kSEED = 1735
kBIAS = "BIAS_CONSTANT"

np.random.seed(kSEED)

class Example:
    """
    Class to represent a document example
    """
    def __init__(self, label, words, vocab):
        """
        Create a new example

        :param label: The label (0 / 1) of the example
        :param words: The words in a list of "word:count" format
        :param vocab: The vocabulary to use as features (list)
        """
        self.nonzero = {}
        self.y = label
        self.x = np.zeros(len(vocab))
        for word, count in [x.split(":") for x in words]:
            if word in vocab:
                assert word != kBIAS, "Bias can't actually appear in document"
                self.x[vocab.index(word)] += float(count)
                self.nonzero[vocab.index(word)] = word
        self.x[0] = 1

def read_dataset(positive, negative, vocab, train_frac=0.9):
    """
    Reads in a text dataset with a given vocabulary

    :param positive: Positive examples
    :param negative: Negative examples
    :param vocab: A list of vocabulary words
    :param test_frac: How much of the data should be reserved for test
    """

    vocab = [x.split("\t")[0] for x in open(vocab, 'r') if '\t' in x]
    assert vocab[0] == kBIAS, \
        "First vocab word must be bias term (was %s)" % vocab[0]

    train_set = []
    test_set = []
    for label, input in [(1, positive), (0, negative)]:
        for line in open(input):
            ex = Example(label, line.split(), vocab)
            if np.random.random() <= train_frac:
                train_set.append(ex)
            else:
                test_set.append(ex)

    # Shuffle the data 
    np.random.shuffle(train_set)
    np.random.shuffle(test_set)

    return train_set, test_set, vocab

In [ ]:
pos_fname = "../data/autos_motorcycles/positive"
neg_fname = "../data/autos_motorcycles/negative"
voc_fname = "../data/autos_motorcycles/vocab"
train_set, test_set, vocab = read_dataset(pos_fname, neg_fname, voc_fname)

In [ ]:
print("The number of words in vocabulary is", len(vocab)-1)

**Explain how the code is creating features (i.e. what text model is being used).** 

The code looks through the positive and negavite files and find the words frequency appeared with each sample in each file, created an arrray with the element reflects the word frequency based on the given vocabulary file. In other words, the code is using the Bag-of-Words Text model.

** Go into the raw text files in the data directory and figure out which label (0/1) refers to which class of document (automobiles or motorcycles)**

label 1 refers to positive, positive raw files refer to motorcycles. label 0 refers to negative, negative raw files refer to automobiles.

### [25 points] Problem 2: Implementing SGD with Lazy Sparse Regularization
***

We've given you a class `LogReg` below which will train a logistic regression classifier to predict whether a document is talking about automobiles or motorcycles. 

**Part A**: In this problem you will modify the `sgd_update` function to perform **unregularized** stochastic gradient descent updates of the weights. Note that you should only update the weights for **non-zero** features, i.e. weights associated with words that appear in the current training example. The code below this cell demonstrates how to instantiate the class and train the classifier.   

We've also given you unit tests in the next cell based on the simple example worked out in  the Lecture 4 in-class notebook.  At first your code will fail both of them. When your code is working you should pass tests called `test_unreg` and `test_learnrate`.  Do not move on to **Part A** until your code passes both of them. 

In [ ]:
class LogReg:
    def __init__(self, train_set, test_set, lam, eta=0.1):
        """
        Create a logistic regression classifier

        :param train_set: A set of training examples
        :param test_set: A set of test examples 
        :param lam: Regularization parameter
        :param eta: The learning rate to use 
        """
        
        # Store training and test sets 
        self.train_set = train_set
        self.test_set = test_set 
        
        # Initialize vector of weights to zero  
        self.w = np.zeros_like(train_set[0].x)
        
        # Store regularization parameter and eta function 
        self.lam = lam
        self.eta = eta
        
        # Create dictionary for lazy-sparse regularization
#         self.last_update = dict()
        self.last_update = dict((elem, 0) for elem in [i for i in range(1, len(self.w))])
        # Make sure regularization parameter is not negative 
        assert self.lam>= 0, "Regularization parameter must be non-negative"
        
        # Empty lists to store NLL and accuracy on train and test sets 
        self.train_nll = []
        self.test_nll = []
        self.train_acc = []
        self.test_acc = []
        
    def sigmoid(self,score, threshold=20.0):
        """
        Prevent overflow of exp by capping activation at 20.
        You do not need to change this function. 

        :param score: A real valued number to convert into a number between 0 and 1
        """

        # if score > threshold, cap value at score 
        if abs(score) > threshold:
            score = threshold * np.sign(score)

        return 1.0 / (1.0 + np.exp(-score)) 

    def compute_progress(self, examples):
        """
        Given a set of examples, compute the NLL and accuracy
        You shouldn't need to change this function. 

        :param examples: The dataset to score
        :return: A tuple of (log probability, accuracy)
        """

        NLL = 0.0
        num_correct = 0
        for ex in examples:
            # compute prob prediction
            p = self.sigmoid(self.w.dot(ex.x))
            # update negative log likelihood
            NLL = NLL - np.log(p) if ex.y==1 else NLL - np.log(1.0-p)
            # update number correct 
            num_correct += 1 if np.floor(p+.5)==ex.y else 0

        return NLL, float(num_correct) / float(len(examples))
    
    def train(self, num_epochs=1, isVerbose=False, report_step=5):
        """
        Train the logistic regression classifier on the training data 

        :param num_epochs: number of full passes over data to perform 
        :param isVerbose: boolean indicating whether to print progress
        :param report_step: how many iterations between recording progress
        """
        iteration = 0
        # Perform an epoch 
        for pp in range(num_epochs):
            # shuffle the data  
            np.random.shuffle(self.train_set)
            # loop over each training example
            for ex in self.train_set:
                # perform SGD update of weights 
                self.sgd_update(ex, iteration)
                # record progress 
                if iteration % report_step == 1:
                    train_nll, train_acc = self.compute_progress(self.train_set)
                    test_nll, test_acc = self.compute_progress(self.test_set)
                    self.train_nll.append(train_nll)
                    self.test_nll.append(test_nll)
                    self.train_acc.append(train_acc)
                    self.test_acc.append(test_acc)
#                     print(self.test_acc)
                    if isVerbose:
                        print("Update {: 5d}  TrnNLL {: 8.3f}  TstNLL {: 8.3f}  TrnA {:.3f}  TstA {:.3f}"
                             .format(iteration-1, train_nll, test_nll, train_acc, test_acc))
                iteration += 1
    
    def sgd_update(self, train_example, iteration):
        """
        Compute a stochastic gradient update to improve the NLL 

        :param train_example: The example to take the gradient with respect to
        :param iteration: The current iteration (an integer)
        """
        
        # TODO implement LSR updates of weights 
        self.w = self.w 
        x = train_example.x
        y = train_example.y

        output = self.sigmoid(self.w.dot(x))
        
        error = output - y
        
        self.w[0] += - self.eta *(x[0] * (error))
        
        for i in range(1, len(x)):
            
            if x[i] != 0 and i != 0:
                
                
                self.w[i] += - self.eta * (x[i] * (error))
                   
# #   lazy sparse regularization
                self.w[i] = self.w[i] * (1 - (2* self.eta * self.lam)) ** (iteration - self.last_update[i] + 1)
                self.last_update[i] = iteration + 1
              
        return self.w
        

In [ ]:
lr = LogReg(train_set, test_set, lam=0, eta=0.1)
lr.train(isVerbose=True)

The unit tests are located in the script `tests.py` in this directory.  Execute the following cell to call the script and run the tests. 

In [ ]:
%run -i tests.py "part A"

**Part B**: After your unregularized updates are working, modify the `sgd_update` function again to perform regularized updates using **Lazy Sparse Regularization**. Note that you should not regularize the bias weight. See the Lecture 4 in-class notebook for a refresher on LSR. **Note**: After implementing LSR, your code should still pass the unit tests for **Part A** when `lam = 0`. 

We've given you a third unit test in the next cell called `test_reg` based on the simple example of LSR worked out in  the Lecture 4 in-class notebook.  Do not move on to **Problem 3** until your code passes the test. 

In [ ]:
%run -i tests.py "part B"

### [10 points] Problem 3: Hyperparameter Tuning 
***

**Part A**: Perform a systematic study of the effect of the regularization parameter on the accuracy of your classifier on the test set.  Which choice of `lam` seems to do the best?  Justify your conclusion with some kind of graphic. 

** Part A Answer**: As the lam gets bigger, the accuracy gets worse. Based on the graph below, the lam of 0 seems to do the best.

In [ ]:
accuracy_lambda = []
for x in range(0,11):
    lr = LogReg(train_set, test_set, x/10, eta=0.1)
    lr.train()
    accuracy_lambda.append(sum(lr.test_acc)/len(lr.test_acc))
    lambda_value = [i for i in range(0,11)]
fig = plt.figure()
plt.axis([0, 11, 0, 1])    
plt.plot(lambda_value, accuracy_lambda,'ro')
plt.xlabel('lambda value')
plt.ylabel('accuracy based on the lambda value')
plt.show()

**Part B**: For the value of `lam` chosen in **Part A** perform a systematic study of the choice of learning rate on the speed of convergence SGD.  Which learning rate seems to give the fastest convergence?  Justify your conclusion with some kind of graphic. 

**Part B Answer** : Based on the graph below, when the learning rate is 0.03, it reaches the convergence with less few iterations than the others. In general, the smaller the learning rate, the faster convergence you will get.

In [ ]:
def plot_speed_SGD(x):
    
    lr = LogReg(train_set, test_set, lam=0, eta=x)
    lr.train()
    lr.test_acc
    iteration_value = [i for i in range(0,len(lr.test_acc))]
    
    plt.axis([0, 220, 0.4, 1])    
    plt.plot(iteration_value, lr.test_acc,'ro')
    plt.xlabel('iteration value')
#     plt.ylabel('accuracy based on the iteration times')
    
plt.figure(figsize=(10,4))
plt.subplot(431); plot_speed_SGD(0.01)
plt.subplot(432); plot_speed_SGD(0.02)
plt.subplot(433); plot_speed_SGD(0.03)
plt.subplot(434); plot_speed_SGD(0.1)
plt.subplot(435); plot_speed_SGD(0.3)
plt.subplot(436); plot_speed_SGD(0.5)
plt.subplot(437); plot_speed_SGD(0.7)
plt.subplot(438); plot_speed_SGD(0.9)
plt.subplot(439); plot_speed_SGD(1.0)

plt.show()

### [10 points] Problem 4: Identifying Predictive and Non-Predictive Words 
***

**Part A**: Find the top 10 words that are the best predictors for each class.  Explain mathematically how you identified them and show any code that you used to find them. 

** Part A answer**  

The best predictors for label 0 is {'heater', 'importing', 'hitting', 'complaints', 'tall', 'detectors', 'equivalent', 'shocked', 'grovel', 'instances'} and for label 1 is {'existing', 'alternative', 'festival', 'prior', 'cheers', 'slack', 'buyer', 'thief', 'hospitals', 'clamps'}. By sorting the weight in ascending and descending order and get the first top ten words for each class.

In [ ]:
def predictor_class(word):
    
    for i in word:
        aa.append(vocab[i])
        from collections import Counter
        cnt = Counter()
        for word in aa:
            cnt[word] += 1
            
    print(cnt.most_common(10))

iteration = 0
aa = []
for ex in train_set:
    weight = lr.sgd_update(ex, iteration)
    word_prediction_label0 = np.argsort(weight)[:10]
    best = predictor_class(word_prediction_label0)
    iteration += 1

In [ ]:
def predictor_class(word):
    
    for i in word:
        aa.append(vocab[i])
        from collections import Counter
        cnt = Counter()
        for word in aa:
            cnt[word] += 1
            
    print(cnt.most_common(10))
iteration = 0
aa = []
for ex in train_set:
    weight = lr.sgd_update(ex, iteration)
    word_prediction_label1 = np.argsort(weight)[-10:]
    best = predictor_class(word_prediction_label1)
    iteration += 1

**Part B**: Find the 10 words that are the worst predictors for class.  Explain mathematically how you identified them and show any code that you used to find them. 

** Part B Answer : ** The worst predictors should be the weights close to zero. So in this case, get the absolute weight value and then the first ten corresponding value from the vocabulary. From the code below, the 10 words are {'crank', 'cochran', 'trash', 'whatsoever', 'girls', 'sakes', 'copied', 'failing', 'hospital', 'spoken'}.

In [ ]:
iteration = 0
aa = []
for ex in train_set:
    weight = lr.sgd_update(ex, iteration)
    word_worst_predictor = np.argsort(abs(weight))[:10]

    for i in word_worst_predictor:
        aa.append(vocab[i])

        from collections import Counter
        cnt = Counter()
        for word in aa:
            cnt[word] += 1
    
    iteration += 1
print(cnt.most_common(10)) 